# Check available LSMS data
In the following code we will check how many countries have a LSMS and which ones have at least two, so we can perform the time travel. For checking this we are using the [search API](https://microdata.worldbank.org/api-documentation/catalog/index.html#operation/CatalogSearch) provided by the World Bank. We search for the keyword (the parameter `sk`) LSMS and specify the country by using the parameter `country`. For the country we use the ISO.

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/src/0_lsms_processing/

/content/drive/MyDrive/src/0_lsms_processing


In [ ]:
from tqdm import tqdm 
import pandas as pd
import requests

from typing import List, Set, Dict

In [ ]:
# Continent of the countries we're searching the lsms for
continent: str = 'asia'
path: str = f'../../data/continents/{continent}/countries_meta/'

In [ ]:
base_url: str = "https://microdata.worldbank.org/index.php/api/catalog/search?sk=lsms&country="

In [ ]:
df: pd.DataFrame = pd.read_csv(path + 'countries_code.csv', index_col=False)

In [ ]:
time_valid: pd.DataFrame = pd.DataFrame() # dataframe with country, iso, year and url for time travel
valid: pd.DataFrame = pd.DataFrame() # dataframe with country, iso, year and url for only one LSMS

for _, country in tqdm(df.iterrows(), total=len(df)):
    result: any = requests.get(base_url + country["iso"]).json()
    result = result["result"]

    if result["found"] == 0: # If not found any lsms for the given country
        continue
    
    count_valid: int = 0 # counter to count valid results
    years: List[str] = [] # save years
    urls: List[str] = [] # save urls
    for res in result["rows"]:

        if len(res["nation"].split(",")) > 1: # only if the country is alone in the `nation` tag, it is the desired data
            continue
        
        years.append(res["year_end"])
        urls.append(res["url"])
        count_valid += 1
    
    if count_valid > 0: # we just need to create a dataframe when, we found something
        tmp_df: pd.DataFrame = pd.DataFrame.from_dict({"name": country["name"], "iso": country["iso"], "year": years, "url": urls})
       
    if count_valid == 1:
        valid = pd.concat([valid, tmp_df])
    
    if count_valid > 1:
        time_valid = pd.concat([time_valid, tmp_df])

In [ ]:
# Save to csv countries with just 1 LSMS
valid.to_csv(path + "countries_lsms_valid.csv", index=False)
# Save to csv countries with 2+ LSMS
time_valid.to_csv(path + "countries_lsms_time_valid.csv", index=False)